# Introduction to AI Agents with Claude

This notebook demonstrates how AI agents work using the Claude API, from basic tool calling to a complete agentic loop.

## What is an Agent?

An AI agent is a system where an LLM can:
1. Decide which tools/functions to call
2. Execute those tools
3. Use the results to take further actions
4. Continue until the task is complete

![](2025-11-05-14-23-10.png)

In [1]:
# Setup
import os
import json
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

## Step 1: Basic Python Functions

First, let's create some simple Python functions that our agent will be able to use.

In [2]:
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # This is a mock function - in reality, you'd call a weather API
    weather_data = {
        "san francisco": "sunny, 72°F",
        "new york": "cloudy, 65°F",
        "london": "rainy, 58°F",
        "tokyo": "clear, 68°F"
    }
    location_lower = location.lower()
    return weather_data.get(location_lower, f"Weather data not available for {location}")

def calculate(operation: str, a: float, b: float) -> float:
    """Perform a mathematical calculation."""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    return operations.get(operation, "Error: Unknown operation")

def create_directory(directory_name: str) -> str:
    """Create a new directory."""
    try:
        os.makedirs(directory_name, exist_ok=True)
        return f"Successfully created directory: {directory_name}"
    except Exception as e:
        return f"Error creating directory: {str(e)}"

# Test our functions
print(get_weather("San Francisco"))
print(calculate("multiply", 15, 7))

sunny, 72°F
105


## Step 2: Define Tools for Claude

Now we need to describe our functions in Claude's tool format. Claude uses a similar but slightly different schema compared to OpenAI.

In [3]:
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name, e.g. San Francisco"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "calculate",
        "description": "Perform a mathematical calculation",
        "input_schema": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The mathematical operation to perform"
                },
                "a": {
                    "type": "number",
                    "description": "The first number"
                },
                "b": {
                    "type": "number",
                    "description": "The second number"
                }
            },
            "required": ["operation", "a", "b"]
        }
    },
    {
        "name": "create_directory",
        "description": "Create a new directory on the file system",
        "input_schema": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create"
                }
            },
            "required": ["directory_name"]
        }
    }
]

# Map function names to actual Python functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate,
    "create_directory": create_directory
}

print(f"Defined {len(tools)} tools for Claude")

Defined 3 tools for Claude


## Step 3: Single Tool Call Example

Let's see how Claude decides to call a tool and how we execute it.

In [5]:
# Create a simple query
messages = [
    {"role": "user", "content": "What's the weather in Tokyo?"}
]

# Send to Claude with tools available
response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=1024,
    messages=messages,
    tools=tools
)

print("Claude Response:")
print(f"Stop reason: {response.stop_reason}")
print(f"Content: {response.content}")

Claude Response:
Stop reason: tool_use
Content: [ToolUseBlock(id='toolu_016VvdKfBx6MsHBbkRemFKPF', input={'location': 'Tokyo'}, name='get_weather', type='tool_use')]


Claude decided to call a tool! Let's execute it and send the result back.

In [7]:
# Check if Claude used a tool
if response.stop_reason == "tool_use":
    # Add the assistant's response to messages
    messages.append({"role": "assistant", "content": response.content})
    
    # Execute each tool call
    tool_results = []
    for block in response.content:
        if block.type == "tool_use":
            tool_name = block.name
            tool_input = block.input
            
            print(f"\nCalling tool: {tool_name}")
            print(f"With input: {tool_input}")
            
            # Get the actual function and call it
            function_to_call = available_functions[tool_name]
            function_response = function_to_call(**tool_input)
            
            print(f"Tool returned: {function_response}")
            
            # Add the tool result
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": block.id,
                "content": str(function_response)
            })
    
    # Add tool results to messages
    messages.append({"role": "user", "content": tool_results})
    
    # Get the final response from Claude
    final_response = client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=1024,
        messages=messages,
        tools=tools
    )
    
    # Extract text from response
    final_text = ""
    for block in final_response.content:
        if hasattr(block, "text"):
            final_text += block.text
    
    print(f"\nFinal Response: {final_text}")
else:
    print("No tool was called")


Calling tool: get_weather
With input: {'location': 'Tokyo'}
Tool returned: clear, 68°F

Final Response: The current weather in Tokyo is clear with a temperature of 68°F (20°C).


## Step 4: Multi-Step Agent Loop

Now let's create a complete agentic loop that can handle multiple tool calls until the task is complete.

In [8]:
def run_agent(user_query: str, max_iterations: int = 5) -> str:
    """
    Run an agentic loop that can make multiple tool calls.
    
    Args:
        user_query: The user's request
        max_iterations: Maximum number of back-and-forth iterations
    
    Returns:
        The final response from the agent
    """
    messages = [{"role": "user", "content": user_query}]
    
    print(f"User: {user_query}\n")
    
    for iteration in range(max_iterations):
        print(f"--- Iteration {iteration + 1} ---")
        
        # Get response from Claude
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=1024,
            messages=messages,
            tools=tools
        )
        
        # If no tool use, we're done
        if response.stop_reason != "tool_use":
            final_text = ""
            for block in response.content:
                if hasattr(block, "text"):
                    final_text += block.text
            print(f"Agent (final): {final_text}\n")
            return final_text
        
        # Add assistant's response to messages
        messages.append({"role": "assistant", "content": response.content})
        
        # Execute all tool calls
        tool_results = []
        for block in response.content:
            if block.type == "tool_use":
                tool_name = block.name
                tool_input = block.input
                
                print(f"Tool call: {tool_name}({tool_input})")
                
                # Execute the function
                function_to_call = available_functions[tool_name]
                function_response = function_to_call(**tool_input)
                
                print(f"Tool response: {function_response}")
                
                # Add tool result
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": block.id,
                    "content": str(function_response)
                })
        
        # Add tool results to messages
        messages.append({"role": "user", "content": tool_results})
        print()
    
    return "Max iterations reached"

# Test with a simple query
result = run_agent("What's the weather in London?")
print(result)

User: What's the weather in London?

--- Iteration 1 ---
Tool call: get_weather({'location': 'London'})
Tool response: rainy, 58°F

--- Iteration 2 ---
Agent (final): The current weather in London is rainy with a temperature of 58°F (about 14°C). You might want to bring an umbrella if you're heading out!

The current weather in London is rainy with a temperature of 58°F (about 14°C). You might want to bring an umbrella if you're heading out!


## Step 5: Complex Multi-Tool Query

Now let's test with a query that requires multiple tool calls.

In [9]:
result = run_agent(
    """What's the weather in San Francisco and New York?
    Then multiply the numbers in their temperatures together."""
)

result

User: What's the weather in San Francisco and New York?
    Then multiply the numbers in their temperatures together.

--- Iteration 1 ---
Tool call: get_weather({'location': 'San Francisco'})
Tool response: sunny, 72°F
Tool call: get_weather({'location': 'New York'})
Tool response: cloudy, 65°F

--- Iteration 2 ---
Tool call: calculate({'operation': 'multiply', 'a': 72, 'b': 65})
Tool response: 4680

--- Iteration 3 ---
Agent (final): **Weather Results:**
- **San Francisco**: Sunny, 72°F
- **New York**: Cloudy, 65°F

**Temperature Multiplication:**
72°F × 65°F = **4,680**



'**Weather Results:**\n- **San Francisco**: Sunny, 72°F\n- **New York**: Cloudy, 65°F\n\n**Temperature Multiplication:**\n72°F × 65°F = **4,680**'

## Step 6: Action-Based Query

Let's test with a query that performs an action (creates a directory).

In [10]:
result = run_agent("Create a folder called 'agent-test-directory'")
result

User: Create a folder called 'agent-test-directory'

--- Iteration 1 ---
Tool call: create_directory({'directory_name': 'agent-test-directory'})
Tool response: Successfully created directory: agent-test-directory

--- Iteration 2 ---
Agent (final): The directory 'agent-test-directory' has been successfully created!



"The directory 'agent-test-directory' has been successfully created!"

## Key Takeaways

1. **Tool Calling**: Claude can decide which tools to call based on the user's query
2. **Tool Execution**: We execute the requested functions and return results to Claude
3. **Agentic Loop**: By repeatedly calling Claude with tool results, we create an agent that can:
   - Break down complex tasks
   - Make multiple tool calls
   - Use results from one tool call to inform the next
   - Continue until the task is complete

### Key Differences from OpenAI:
- Claude uses `input_schema` instead of `parameters` for tool definitions
- Tool responses are sent back as `tool_result` content blocks
- Claude's `stop_reason` indicates when tools are used
- Response content is a list of blocks (text and tool_use blocks)

This is the foundation of how AI agents work - they bridge the gap between language understanding and taking real actions!